# 使用DMT训练框架

通过以下4步，使用DMT框架训练并使用你自己的深度学习模型：
1. 定义数据集
2. 定义模型
3. 重写训练器和测试方法
4. 编写配置文件

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from easydict import EasyDict
from torch.utils.data import Dataset

from src.trainers import Trainer

## 1. 定义数据集

DMT框架使用`torch.utils.data.Dataset`作为数据集的基类，唯一的限制是你只能通过传入一个`EasyDict`实例`config`参数来实例化数据集。

例如在`example`中，我们合成一个0到1之间的二维数据集`exampleDataset`来并完成在其上的分类任务。下面是一个简化版本：
- label = 0 if x > 0.5 and y > 0.5
- label = 1 if $x^2 + y^2 < 0.5$
- label = 2 otherwise

In [ ]:
class ExampleDataset(Dataset):
    def __init__(self, args: EasyDict):
        self.num_classes = 3
        assert hasattr(args, 'size'), "size should be specified when data_path is not specified"
        self.size = args.size
        seed = args.seed if hasattr(args, 'seed') else None 
        self.data = self._generate_data(seed)
        self.label = self._calculate_label()
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx: int):
        return self.data[idx], self.label[idx]
    
    def _generate_data(self, seed: int = None):
        if seed is not None:
            torch.manual_seed(seed)
        return torch.rand(self.size, 2)
    
    def _calculate_label(self):
        label_0 = ((self.data[:, 0] > 0.5) & (self.data[:, 1] > 0.5)).long()
        label_1 = (self.data[:, 0] ** 2 + self.data[:, 1] ** 2 < 0.5).long()
        return 2 - 2 * label_0 - label_1

数据集可视化：

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

dataset = ExampleDataset(EasyDict(size=1000, seed=42))

cmap = {0: "r", 1: "g", 2: "b"}
fig, ax = plt.subplots()
data, label = dataset[:]
x = data[:, 0].clone()
y = data[:, 1].clone()
label = label.clone().tolist()
ax.scatter(x, y, c=[cmap[l] for l in label], marker=".")
plt.show()

## 2. 定义模型

DMT框架使用`torch.nn.Module`作为模型的基类，同样的，你只能通过传入一个`EasyDict`实例`config`参数来实例化模型。

我们使用一个简单的mlp来完成这个分类任务：

In [ ]:
class MLP(nn.Module):
    def __init__(self, config: EasyDict):
        super().__init__()
        hidden_size = config.hidden_size
        self.fc1 = nn.Linear(2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 3)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

## 3. 重写训练器和测试方法

DMT框架使用自定义的`Trainer`作为训练器的基类，在使用之前，你需要继承`Trainer`并重写`_compute_loss`方法。

至此训练模型的准备工作已经完成，但是为了监控训练过程中其他指标（如测试指标）则需要重写`_eval_epoch`方法，从而达到在训练过程中进行评估的目的。

In [ ]:
class ExampleTrainer(Trainer):
    def __init__(self, config: EasyDict):
        super().__init__(config)

    def _compute_loss(self, inputs, targets):
        outputs = self.model(inputs)
        return F.cross_entropy(outputs, targets)
    
    def _eval_epoch(self, epoch):
        pass #* calculate accuracy on evaluation dataset

## 4. 编写配置文件

DMT框架使用 *.yml* 文件对实验进行配置，配置文件分为7个部分：
1. 实验信息：包括名称、os、gpu等信息
2. 模型参数
3. 数据集参数
4. 训练过程参数：包括优化器、学习率、训练轮数等
5. 评估过程参数：包括评估批大小等
6. 采样过程参数
7. 其他参数

具体可参考`base_config.yml`文件。

# 启动DMT框架

以上步骤中的具体代码实现均为简化版本，完整代码可参考`src_example`文件夹中的内容。下面我们给出使用DMT框架训练模型的CMD命令：

In [ ]:
!python src_example/main.py --config example --train

multi-gpu evaluate

In [ ]:
!CUDA_VISIBLE_DEVICES="0,1" accelerate launch --num_processes=2 src_example/main.py --config example --eval

Tensorboard 可视化训练过程

In [ ]:
!tensorboard --logdir experiments/MLP-ExampleDataset/logs --port=8008